<a href="https://colab.research.google.com/github/Rigved19/Covid-Classification-Using-X-Ray-Images/blob/main/Covid_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install Pillow

In [1]:
#Mounting Google Drive to Colab NoteBook
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Extracting all files from the zip file
from zipfile import ZipFile
file_name = "/content/drive/My Drive/CovidDataset (1).zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print("Completed")

Completed


###Building CNN Model

*  Convolution Layer
*  Pooling Layer
*  Artificial Neural Network


In [3]:
#Importing all layers and models from Keras 
from tensorflow.keras.layers import *
from tensorflow.keras.models import * 
import tensorflow.keras as tf

In [4]:
#Training model
model = Sequential()   ## Creating a blank model

#Adding Convolution Layer with 32 feauture detectors and reshaping input to specified size
#Adding Pooling Layer
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))  
model.add(MaxPooling2D(pool_size=(2,2)))

#Addition of Convolutional and Pooling Layers (Optimum number found by trial and error)
model.add(Conv2D(64,(3,3),activation='relu'))  #Using double no. of feature detectors
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))                       #Reduce the overfitting
 
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
 
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#ANN Network starts 
model.add(Flatten())                      #Input Layer
model.add(Dense(64,activation='relu'))    #Hidden Layer of ANN
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))   #Output Layer

#Since it's a binary classification problem 
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy']) 

##Image Preprocessing and Data Augmentation

In [8]:
#Image Preprocessing AND Daata Augmentation
from tensorflow.keras.preprocessing import image
#Rescale -Similar to Normalization [0,255) --> [0,1]
#Shear Range -Creates rotated images
#Zoom Range -Creates zoomed images

train_datagen = image.ImageDataGenerator(rescale = 1./255, shear_range = 0.2,zoom_range = 0.2, horizontal_flip = True) #Training Instance
 
test_dataset = image.ImageDataGenerator(rescale=1./255)  #Validation Instance

###Creating training and validation images

In [6]:
#Reshaping Training and Validation images

#Creating training images
train_generator = train_datagen.flow_from_directory(     
    '/content/CovidDataset/Train',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary')

#Creating testing images
validation_generator = test_dataset.flow_from_directory(
    '/content/CovidDataset/Val',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary')

Found 224 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


Training the Model 

In [7]:
#Train the model

#Printing test and validation loss and accuracy at the same time
history = model.fit_generator(
    train_generator,
    steps_per_epoch=7,
    epochs = 20,
    validation_data = validation_generator,
    validation_steps=1
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
7/7 [==============================] - 40s 1s/step - loss: 0.7185 - accuracy: 0.5625 - val_loss: 0.6916 - val_accuracy: 0.5312
Epoch 2/20
7/7 [==============================] - 8s 1s/step - loss: 0.6806 - accuracy: 0.5446 - val_loss: 0.6817 - val_accuracy: 0.5625
Epoch 3/20
7/7 [==============================] - 8s 1s/step - loss: 0.6215 - accuracy: 0.6964 - val_loss: 0.5868 - val_accuracy: 0.9062
Epoch 4/20
7/7 [==============================] - 8s 1s/step - loss: 0.4713 - accuracy: 0.7634 - val_loss: 0.3368 - val_accuracy: 0.9688
Epoch 5/20
7/7 [==============================] - 8s 1s/step - loss: 0.3172 - accuracy: 0.8616 - val_loss: 0.1863 - val_accuracy: 0.9688
Epoch 6/20
7/7 [==============================] - 8s 1s/step - loss: 0.3417 - accuracy: 0.8973 - val_loss: 0.2630 - val_accuracy: 0.9062
Epoch 7/20
7/7 [==============================] - 8s 1s/step - loss: 0.2617 - accuracy: 0.8929 - val_loss: 0.1465 - val_accuracy: 0.9688
Epoch 8/20
7/7 [========================

Testing Model with random X-Ray image from net

In [25]:
from tensorflow.keras.preprocessing import image
from PIL import Image
import numpy as np
img = image.load_img('/content/test_image.jpeg',target_size=(224,224)) #Loading Image 
img = image.img_to_array(img)      # 
img = np.expand_dims(img,axis=0)   # Flattening
ypred = model.predict(img)
if ypred[0][0] == 1:
  print("Covid Negative")
else:
  print("Covid Positive")


Covid Positive
